In [1]:
from __future__ import unicode_literals, print_function, division
import unicodedata
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Seq2Seq

In [ ]:
class EncoderRNN(nn.Module):

    def __init__(self,input_size,hidden_size,dropout_p=0.1):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden


In [ ]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [ ]:
class DecoderRNN(nn.Module):

    def __init__(self,hidden_size,output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size,batch_first=True)
        self.out = nn.Linear(hidden_size,output_size)

    def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size,1,dtype=torch.long,device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing
                decoder_input = target_tensor[:,i].unsqeeze(1)
            
            else:
                _,topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs,dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs,dim=-1)
        return decoder_outputs, decoder_hidden, None

    # single node forward
    def forward_step(self,input,hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output,hidden)
        output = self.out(output)
        return output, hidden

Attention Decoder Bahdanau